# Assignment 8 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [15]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler;

## Part [a]

In [16]:
imgs = np.load("mnist.npz")

#Flatten images
X_train = imgs['x_train'].reshape((60000, 28 * 28))
X_test = imgs['x_test'].reshape((10000, 28 * 28))

#scale their features
scale = StandardScaler().fit(X_train)
X_train_scaled = scale.transform(X_train)
X_test_scaled = scale.transform(X_test)

## Part [b]

## Part [c]

## Part [d]